**Install Pyspark**

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

**Reading the csv file**

In [3]:
spark = SparkSession.builder.appName("SuperstoreSalesAnalysis").getOrCreate()

In [4]:
df = spark.read.csv("/content/Sample_Superstore_Cleaned (1).csv", header=True, inferSchema=True)

Display rows of the dataset

In [5]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|2016-11-08|2016-11-11|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

Display data type of each column

In [6]:
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



Display Column Names

In [7]:
df.columns

['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

Count Number of Rows and Columns of the Dataset

In [8]:
#rows
df.count()

9994

In [9]:
#Columns
len(df.columns)

21

Get overall Statistics About the dataset

In [10]:
df.describe().show()

+-------+------------------+--------------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+------------------+------------------+------------------+------------------+
|summary|            Row ID|      Order ID|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|    City|  State|       Postal Code| Region|     Product ID|  Category|Sub-Category|        Product Name|             Sales|          Quantity|          Discount|            Profit|
+-------+------------------+--------------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+------------------+------------------+------------------+------------------+
|  count|              9994|          9994|          9994|       9994|              9994|       9994|        

**Descriptive Analysis**

1. How many unique customers, products, and orders are there?

In [11]:
from pyspark.sql.functions import to_date
df = df.withColumn("Order Date", to_date(df["Order Date"], "MM/dd/yyyy")) \
.withColumn("Ship Date", to_date(df["Ship Date"], "MM/dd/yyyy"))
df.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



2. What is the total sales, profit, and quantity sold?

In [12]:
from pyspark.sql.types import DoubleType

df = df.withColumn("Sales", df["Sales"].cast(DoubleType())) \
       .withColumn("Quantity", df["Quantity"].cast(DoubleType())) \
       .withColumn("Discount", df["Discount"].cast(DoubleType()))

# Perform aggregation
aggregated_df = df.agg(F.sum("Sales").alias("TotalSales"),
                        F.sum("Profit").alias("TotalProfit"),
                        F.sum("Quantity").alias("TotalQuantity"))

# Collect and print the results
results = aggregated_df.collect()[0]
total_sales = results["TotalSales"]
total_profit = results["TotalProfit"]
total_quantity = results["TotalQuantity"]

print(f"Total Sales: {total_sales}")
print(f"Total Profit: {total_profit}")
print(f"Total Quantity Sold: {total_quantity}")

Total Sales: 2272449.8562999545
Total Profit: 285707.60220000165
Total Quantity Sold: 58134.36199999999


3. Which are the top 10 most sold products by sales and by quality?

In [13]:
from pyspark.sql.functions import sum, desc, col

df.groupBy("Product Name").sum("Sales").withColumnRenamed("sum(Sales)", "Total Sales").orderBy(desc("Total Sales")).show(10)

+--------------------+------------------+
|        Product Name|       Total Sales|
+--------------------+------------------+
|Canon imageCLASS ...|         61599.824|
|Fellowes PB500 El...|         27453.384|
|Cisco TelePresenc...|          22638.48|
|HON 5400 Series T...|         21870.576|
|GBC DocuBind TL30...|19823.479000000003|
|GBC Ibimaster 500...|           19024.5|
|Hewlett Packard L...|         18839.686|
|"HP Designjet T52...|         18374.895|
|GBC DocuBind P400...|         17965.068|
|High Speed Automa...|17030.311999999998|
+--------------------+------------------+
only showing top 10 rows



4. What is the average discount given?

In [14]:
from pyspark.sql.functions import avg

df.agg(avg("Discount")).show()

+------------------+
|     avg(Discount)|
+------------------+
|0.3155949113492862|
+------------------+



**Sales and Profit Analysis**

5. Which category generates the highest sales and profit?

In [15]:
df.groupBy("Category").agg(sum("Sales").alias("TotalSales"),sum("Profit").alias("TotalProfit")).orderBy(desc("TotalSales")).show()

+---------------+-----------------+------------------+
|       Category|       TotalSales|       TotalProfit|
+---------------+-----------------+------------------+
|     Technology|835900.0669999964|145388.29659999989|
|      Furniture|733046.8612999996| 19686.42720000003|
|Office Supplies|703502.9280000031|120632.87839999991|
+---------------+-----------------+------------------+



6. Which sub-category is most profitable, and which is least profitable?

In [16]:
df.groupBy("Sub-Category").sum("Profit").orderBy(desc("sum(Profit)")).show()

+------------+-------------------+
|Sub-Category|        sum(Profit)|
+------------+-------------------+
|     Copiers|  55617.82490000001|
|      Phones|         44449.0791|
| Accessories|  41936.63569999993|
|       Paper|  32795.56099999999|
|     Binders| 30038.821299999996|
|      Chairs| 26590.166300000026|
|     Storage|         21529.9083|
|  Appliances| 18138.005399999995|
| Furnishings| 14294.297999999995|
|         Art|  6527.786999999998|
|   Envelopes|  6461.269100000003|
|      Labels|  5546.253999999998|
|    Machines|          3384.7569|
|   Fasteners|  942.6377999999997|
|    Supplies|-1347.3654999999983|
|   Bookcases|-3472.5559999999978|
|      Tables|-17725.481100000008|
+------------+-------------------+



7. Which **States and Cities** contribute the most to overall profit?

In [17]:
df.groupBy("State").sum("Profit").orderBy(desc("sum(Profit)")).show(10)
df.groupBy("City").sum("Profit").orderBy(desc("sum(Profit)")).show(10)

+----------+------------------+
|     State|       sum(Profit)|
+----------+------------------+
|California| 75473.84360000011|
|  New York| 73944.56520000006|
|Washington|33200.475099999996|
|  Michigan|24340.948699999994|
|  Virginia|        18461.0153|
|   Indiana|18344.437500000004|
|   Georgia|        16529.0269|
|  Kentucky|11199.696600000005|
| Minnesota|        10823.1874|
|  Delaware| 9836.424200000001|
+----------+------------------+
only showing top 10 rows

+-------------+------------------+
|         City|       sum(Profit)|
+-------------+------------------+
|New York City|        62061.2982|
|  Los Angeles|30404.695899999973|
|      Seattle|28894.395700000005|
|San Francisco|17179.468499999988|
|      Detroit|13117.051600000003|
|    Lafayette|        10001.5463|
|      Jackson|         7552.6813|
|      Atlanta|         6993.6629|
|  Minneapolis| 6824.584599999999|
|     Columbus| 6380.506399999996|
+-------------+------------------+
only showing top 10 rows



8. Which product or categories are sold at a loss?

In [18]:
df.groupBy("Product Name").sum("Profit").filter(col("sum(Profit)") < 0).orderBy("sum(Profit)").show(10)

+--------------------+-------------------+
|        Product Name|        sum(Profit)|
+--------------------+-------------------+
|Cubify CubeX 3D P...|         -8879.9704|
|Lexmark MX611dhe ...|          -4589.973|
|Cubify CubeX 3D P...|         -3839.9904|
|Chromcraft Bull-N...|         -2876.1156|
|Bush Advantage Co...|-1934.3975999999998|
|GBC DocuBind P400...|-1878.1662000000003|
|Cisco TelePresenc...|         -1811.0784|
|Martin Yale Chadl...|         -1299.1836|
|Balt Solid Wood R...|         -1201.0581|
|BoxOffice By Desi...|         -1148.4375|
+--------------------+-------------------+
only showing top 10 rows



**Customer Analysis**

9. Who are the top 10 customer by sales contribution?

In [19]:
df.groupBy("Customer Name").sum("Sales").orderBy(desc("sum(Sales)")).show(10)

+------------------+------------------+
|     Customer Name|        sum(Sales)|
+------------------+------------------+
|       Sean Miller|          25043.05|
|      Tamara Chand|19017.847999999998|
|      Raymond Buch|         15117.339|
|      Tom Ashbrook|          14595.62|
|     Adrian Barton|14355.610999999997|
|      Sanjit Chand|14142.333999999999|
|      Ken Lonsdale|         14071.917|
|      Hunter Lopez|12873.297999999999|
|      Sanjit Engle|12209.438000000002|
|Christopher Conant|         12129.072|
+------------------+------------------+
only showing top 10 rows



10. Which customer segment (Consumer, Corporate, Home office) is the most profitable?

In [20]:
df.groupBy("Segment").agg(sum("Sales"),sum("Profit")).show()

+-----------+------------------+-----------------+
|    Segment|        sum(Sales)|      sum(Profit)|
+-----------+------------------+-----------------+
|   Consumer|1150166.1819999903|134443.8591999996|
|Home Office| 425679.1605000003|59833.77810000006|
|  Corporate| 696604.5138000002|91429.96490000018|
+-----------+------------------+-----------------+



11. How many repeat vs. new customers are there each year?

In [21]:
from pyspark.sql.functions import year, countDistinct

df.groupBy(year("Order Date").alias("Year"), "Customer ID").count()\
.groupBy("Year").agg(countDistinct("Customer ID")).show()

+----+---------------------------+
|Year|count(DISTINCT Customer ID)|
+----+---------------------------+
|2015|                        573|
|2014|                        595|
|2016|                        638|
|2017|                        693|
+----+---------------------------+



**Shipping & Order Analysis**

12. What is the average shipping delay (Ship Date- Order Date)?

In [22]:
from pyspark.sql.functions import datediff, avg

df.withColumn("Shipping_Days", datediff("Ship Date", "Order Date")).agg(avg("Shipping_Days")).show()

+------------------+
|avg(Shipping_Days)|
+------------------+
| 3.958174904942966|
+------------------+



13. Which shipping mode is used the most, and which gives higher profit margins?

In [23]:
from pyspark.sql.functions import count, sum, desc, col

df.groupBy("Ship Mode").agg(count("*").alias("Total Orders"),sum("Profit").alias("Total Profit")).show()

+--------------+------------+------------------+
|     Ship Mode|Total Orders|      Total Profit|
+--------------+------------+------------------+
|   First Class|        1538| 48788.54900000001|
|      Same Day|         543|16110.266399999986|
|  Second Class|        1945| 56724.30419999997|
|Standard Class|        5968|164084.48259999973|
+--------------+------------+------------------+



14. Is there any correlation between shipping time and customer segment?

In [24]:
df.withColumn("Shipping_Days",datediff("Ship Date","Order Date")).groupBy("Segment").avg("Shipping_Days").show()

+-----------+------------------+
|    Segment|avg(Shipping_Days)|
+-----------+------------------+
|   Consumer| 3.941822384896937|
|Home Office|3.9220415030846887|
|  Corporate| 4.007615894039735|
+-----------+------------------+



**Regional & Temporal Analysis**

15. How do sales and profit vary across the four region (East, West, North, South)?

In [25]:
df.groupBy("Region").agg(sum("Sales"),sum("Profit")).show()

+-------+------------------+------------------+
| Region|        sum(Sales)|       sum(Profit)|
+-------+------------------+------------------+
|  South|388983.58500000037|46650.341000000044|
|Central| 497800.8728000007| 40150.50299999999|
|   East| 672194.0539999981| 91603.05670000015|
|   West| 713471.3445000004|107303.70150000004|
+-------+------------------+------------------+



16. What are the monthly and yearly sales trends?

In [27]:
from pyspark.sql.functions import year, month

df.groupBy(year("Order Date").alias("Year"),month("Order Date").alias("Month")).sum("Sales").orderBy("Year","Month").show()

+----+-----+------------------+
|Year|Month|        sum(Sales)|
+----+-----+------------------+
|2014|    1|14161.348999999998|
|2014|    2|          4119.816|
|2014|    3| 55526.19900000002|
|2014|    4|28139.560999999994|
|2014|    5|         23634.667|
|2014|    6| 34508.99560000003|
|2014|    7| 33500.87299999999|
|2014|    8| 27603.51249999999|
|2014|    9| 81496.80679999998|
|2014|   10|31394.940999999988|
|2014|   11| 78297.24069999997|
|2014|   12| 69379.83649999999|
|2015|    1|18085.115599999994|
|2015|    2|11924.271999999999|
|2015|    3|38621.291999999994|
|2015|    4|32640.482499999987|
|2015|    5|29325.970500000003|
|2015|    6|24659.684000000005|
|2015|    7| 28524.52099999999|
|2015|    8|36380.928199999995|
+----+-----+------------------+
only showing top 20 rows



17. Which year was the most profitable overall?

In [29]:
df.groupBy(year("Order Date").alias("Year")).sum("Profit").orderBy(desc("sum(Profit)")).show()

+----+-----------------+
|Year|      sum(Profit)|
+----+-----------------+
|2017|93642.64579999988|
|2016|81247.20840000016|
|2015|61756.92170000004|
|2014|49060.82630000002|
+----+-----------------+



18. What are the sessional trend (e.g., highest sales month)?

In [31]:
df.groupBy(month("Order Date").alias("Month")).sum("Sales").orderBy("Month").show()

+-----+------------------+
|Month|        sum(Sales)|
+-----+------------------+
|    1| 94539.34159999999|
|    2| 59012.82540000001|
|    3|203719.26379999987|
|    4|135387.35759999996|
|    5| 153513.3096999999|
|    6|151039.43330000006|
|    7| 145623.8500000002|
|    8|157642.25099999984|
|    9| 303536.6657000002|
|   10| 198440.0027000002|
|   11| 348834.5570000001|
|   12| 321160.9985000003|
+-----+------------------+



19. What is the imapct of discounts on profit? (e.g., is higher discount reduces profit)

In [32]:
df.groupBy("Discount").avg("Profit").orderBy("Discount").show()

+--------+-------------------+
|Discount|        avg(Profit)|
+--------+-------------------+
|    NULL|  251.7498181818182|
|     0.0|  68.10909261327028|
|     0.1|  96.05507446808508|
|    0.15| 27.288298076923077|
|     0.2| 25.198563349309303|
|     0.3|-45.679636123348004|
|    0.32| -88.56065555555558|
|     0.4|-112.51435121951225|
|    0.45|-226.64646363636365|
|     0.5|  -310.703456060606|
|     0.6| -42.03333030303031|
|     0.7|  -101.390593606138|
|     0.8|-105.74120982456134|
|     1.0| 0.1272727272727273|
|     2.0|0.17761194029850744|
|     3.0|0.18695652173913035|
|     4.0|  0.175609756097561|
|     5.0|0.19090909090909092|
|     6.0|0.17058823529411765|
|     7.0|0.24000000000000002|
+--------+-------------------+
only showing top 20 rows



20. Which product should the company stop selling due to consistent losses?

In [33]:
df.groupBy("Product Name").sum("Profit").filter(col("sum(Profit)")< 0).orderBy("sum(Profit)").show(20)

+--------------------+-------------------+
|        Product Name|        sum(Profit)|
+--------------------+-------------------+
|Cubify CubeX 3D P...|         -8879.9704|
|Lexmark MX611dhe ...|          -4589.973|
|Cubify CubeX 3D P...|         -3839.9904|
|Chromcraft Bull-N...|         -2876.1156|
|Bush Advantage Co...|-1934.3975999999998|
|GBC DocuBind P400...|-1878.1662000000003|
|Cisco TelePresenc...|         -1811.0784|
|Martin Yale Chadl...|         -1299.1836|
|Balt Solid Wood R...|         -1201.0581|
|BoxOffice By Desi...|         -1148.4375|
|Riverside Furnitu...|            -1147.4|
|Epson TM-T88V Dir...|           -1057.23|
|Hon 2090 Pillow ...| -989.0495999999999|
|O'Sullivan 4-Shel...|          -975.0988|
|Bretford Just In...|           -964.194|
|Zebra GK420t Dire...|            -938.28|
|3.6 Cubic Foot Co...|          -872.0752|
|Bevis Oval Confer...|          -856.0144|
|Tennsco Single-Ti...| -825.7479999999999|
|BPI Conference Ta...| -795.9725000000001|
+----------

21. Can we identify cross-sell opportunities(products often bought together)?

In [35]:
df.groupBy("Order ID", "Sub-Category").count().groupBy("Sub-Category").count().orderBy(desc("count")).show()

+------------+-----+
|Sub-Category|count|
+------------+-----+
|     Binders| 1316|
|       Paper| 1191|
| Furnishings|  877|
|      Phones|  814|
|     Storage|  777|
|         Art|  731|
| Accessories|  718|
|      Chairs|  576|
|  Appliances|  451|
|      Labels|  346|
|      Tables|  307|
|   Envelopes|  249|
|   Bookcases|  224|
|   Fasteners|  215|
|    Supplies|  187|
|    Machines|  112|
|     Copiers|   68|
+------------+-----+



22. Which cities have high sales but low profit margins?

In [36]:
df.groupBy("City").agg(sum("Sales").alias("Total Sales"),sum("Profit").alias("Total Profit")).filter(col("Total Sales") >5000).orderBy("Total Sales").show(20)

+--------------+------------------+-------------------+
|          City|       Total Sales|       Total Profit|
+--------------+------------------+-------------------+
|    Morristown| 5033.099999999999| 1838.2270999999998|
|     Brentwood| 5090.113999999999|          1216.8104|
|         Tampa| 5154.704500000002|-329.69140000000004|
|       Midland|          5291.512| 2293.6827999999996|
|       Madison| 5346.790000000002| 1123.6552000000004|
|    Des Moines|          5378.798|          1571.0387|
|        Mobile|           5462.99|          2175.8292|
|    Alexandria| 5483.450000000001| 300.94710000000003|
|       Clinton|          5514.251| 1274.2421000000006|
|         Tulsa|           5556.97| 1506.9349999999995|
|      Lakeland|         5595.0625|          -189.4711|
|       Everett|5665.7339999999995|          1361.4787|
|Virginia Beach|           5752.13|          2024.8837|
|        Tucson| 5827.759999999999| 142.76539999999994|
|      San Jose| 5865.308999999999| 1213.9986999